# zerion_api

> Endpoints for the `zerion` api.

In [ ]:
#|default_exp zerion_api

In [ ]:
#| hide
from nbdev.showdoc import *

**Functionality:**

`get_portfolio_value`

Get the value of an address in USD. At the moment just at latest time.

`get_wallet_positions`

Get the portfolio of an address (i.e. all the positions). At the moment just at latest time.

`get_chart`

Can use to get the historical prices of an asset.

`token_address_to_prices`

Use to extract a dictionary of `dates:prices` given an address.

In [ ]:
#| export

import json
import requests
import os
from dotenv import load_dotenv
load_dotenv()

ZERION_AUTH = os.environ.get('zerion_auth')

In [ ]:
#| hide

_address = '0x7364a0f792e073814B426c918bf72792575b6c18'

In [ ]:
#| export

def get_portfolio_value(address:str):
    """Get the total value of a portfolio in USD"""
    #TODO: Obviously need to complete docstring etc, but we can leave as vague for now; unclear if we will
    #e.g. wrap in class etc.

    #API docs: https://developers.zerion.io/reference/getwalletportfolio

    #COMMENT: note that the docs have some nice info about different responses, e.g. 202 etc. This may be useful later on.

    url = f"https://api.zerion.io/v1/wallets/{address}/portfolio?currency=usd"

    headers = {
        "accept": "application/json",
        "authorization": ZERION_AUTH
    }

    response = requests.get(url, headers=headers)

    return response.json()

# if __name__ == '__main__':
#     _dict = get_portfolio_value(_address)
#     print(f"Portfolio value according to zerion is: {_dict['data']['attributes']['positions_distribution_by_type']['wallet']}")


Portfolio value according to zerion is: 141.11988657032236


Kinda complicated to key-index in, but we can use for debugging etc. Anyway...

Next, get list of wallets positions:

In [ ]:
#| export

def get_wallet_positions(address:str, 
                         position_types:list[str]=None,
                         protocol_ids:list[str]=None,
                         fungible_ids:list[str]=None,
                         currency:str='usd'):
    """This endpoint returns a list of wallet positions.
    
        API docs: https://developers.zerion.io/reference/listwalletpositions
    """ 

    position_types_str = ''
    protocol_ids_str = ''
    fungible_ids_str = ''
    
    if position_types:
        position_types_str = f"filter[position_types]={','.join(position_types)}&"
    if protocol_ids:
        protocol_ids_str = f"filter[protocol_ids]={','.join(protocol_ids)}&"
    if fungible_ids:
        fungible_ids_str = f"filter[fungible_ids]={','.join(fungible_ids)}&"

    url = f"https://api.zerion.io/v1/wallets/{address}/positions/?currency={currency}&{position_types_str}{protocol_ids_str}{fungible_ids_str}filter[trash]=only_non_trash&sort=value"

  
    headers = {
        "accept": "application/json",
        "authorization": ZERION_AUTH
              }

    response = requests.get(url, headers=headers)


    return response.json() 

# #This is mostly to help prototyping below, perhaps we will put its own cell explaining "how to use" eventually
# if __name__ == '__main__':

#     _dict = get_wallet_positions(_address)
#     _portfolio = [_dict['data'][i]['attributes'] for i in range(len(_dict['data']))]
#     print(f"First item in portfolio is: {_portfolio[0]}")
#     print('Note that there is other information in the _dict object:\n')
#     for k,v in _dict['data'][0].items():
#         print(f"\tKey type is: {type(k)}, Value type is: {type(v)}")
#         print(f"\tKey: {k}, Value: {v}\n")



Alright, let's work out how to extract the data that we need:

In [ ]:
# print(f"The name of first guy in portfolio is: {_portfolio[0]['fungible_info']['name']}")
# print(f"The symbol of first guy in portfolio is: {_portfolio[0]['fungible_info']['symbol']}")
# print(f"The value of first guy in portfolio is: {_portfolio[0]['value']}")

In [ ]:
# #Simple test: summing portfolio value should get same result as earlier call

# none_to_zero = lambda x: 0 if x is None else x

# sum([none_to_zero(asset['value']) for asset in _portfolio])

# #Ok, it is fairly close, but not exactly the same. A bit annoying but ok for now...
# #1122.864752311196

Get "chart" for fungible asset

In [ ]:
#| export

def get_chart(address:str,period:str='max',currency:str='usd'):
    """The 'charts' endpoint in the Zerion API allows users to retrieve a chart for a specific fungible asset.
        `period` may be any of: 'max','hour', 'day', 'week', 'month', 'year'.

        API docs: https://developers.zerion.io/reference/getfungiblechart
    
    """

    url = f"https://api.zerion.io/v1/fungibles/{address}/charts/{period}?currency={currency}"

    headers = {
        "accept": "application/json",
        "authorization": ZERION_AUTH
              }


    response = requests.get(url, headers=headers)
    return response.json()



In [ ]:
#from api_endpoints.etherscan_api import *
#blockfetcher = BlockFetcher(etherscan_api_key=etherscan_api_key)
#blockfetcher.get_block_for_date(_dict['data']['attributes']['begin_at'].split('T')[0])

We now have historical prices and can extract:

In [ ]:
#| export

import datetime
from collections import defaultdict

def unix_timestamp_to_date(unix_timestamp):
    return datetime.datetime.utcfromtimestamp(unix_timestamp).strftime('%Y/%m/%d')

def timestamp_price_pairs_to_date_dict(points:list)->dict:
    """Inputs:
            points: list of lists (pairs) where first coord is unix timestamp, second is price on that date.
                    e.g. [1683788578, 0.00010605266543295529] 
       Outputs:
                    dict where key is date string and value is price on that date.
       Comment:
                    The source code is basically self explanatory.      
    """

    return {unix_timestamp_to_date(point[0]): point[1] for point in points}

def timestamp_price_pairs_to_date(points:list)->list:
    """Inputs:
            points: list of lists (pairs) where first coord is unix timestamp, second is price on that date.
                    e.g. [1683788578, 0.00010605266543295529] 
       Outputs:
                    list of lists (pairs) where first coord is date string, second is price on that date.
                    e.g. ['2023/05/11', 0.00010605266543295529] 
       Comment:
                    The source code is basically self explanatory.      
    """

    return [[unix_timestamp_to_date(point[0]), point[1]] for point in points]

def timestamp_price_pairs_to_avg_date_dict(points: list) -> dict:
    """Inputs:
            points: list of lists (pairs) where first coord is unix timestamp, second is price on that date.
                    e.g. [1683788578, 0.00010605266543295529] 
       Outputs:
                    dict where key is the date string and value is the average price for that date.
        
    """
    
    date_price_aggregator = defaultdict(lambda: {'sum': 0, 'count': 0})

    for point in points:
        date = unix_timestamp_to_date(point[0])
        date_price_aggregator[date]['sum'] += point[1]
        date_price_aggregator[date]['count'] += 1

    return {date: data['sum']/data['count'] for date, data in date_price_aggregator.items()}


def token_address_to_prices(address:str,period:str='max',currency:str='usd')->dict:
    """Wrapper function to get prices for a token address.
        Inputs: 
                address: str, address of token
        Outputs:
                dict where key is date string and value is price on that date.

        #TODO: This gets ALL the prices since origin to the current date. I guess we want to save this so
          we can cache and save on API calls. 
    """

    _points = get_chart(address=address,period=period,currency=currency)['data']['attributes']['points']
    date_to_price_dict = timestamp_price_pairs_to_date_dict(_points)

    return date_to_price_dict


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()